### Fast Online Distributed Matrix Completion

#### 1. Read and prepare the data

In [1]:
rawRatings = sc.textFile('/mnt/data/Downloads/ml-latest/ratings.csv')

In [2]:
def get_ratings_tuple(entry):
    """ Parse a line in the ratings dataset
    Args:
        entry (str): a line in the ratings dataset in the form of UserID::MovieID::Rating::Timestamp
    Returns:
        tuple: (UserID, MovieID, Rating)
    """
    items = entry.split(',')
    return int(items[0]), int(items[1]), float(items[2])

In [3]:
ratingsRDD = rawRatings.map(get_ratings_tuple).cache()

In [4]:
trainingRDD, validationRDD, testRDD = ratingsRDD.randomSplit([6, 2, 2], seed=0L)

In [13]:
def mask(entry):
    return entry[0], entry[1], 1

In [14]:
maskTrainingRDD = trainingRDD.map(mask).cache()
maskValidationRDD = validationRDD.map(mask).cache()
maskTestRDD = testRDD.map(mask).cache()

#### Normalize

In [18]:
def max(a, b):
    if a > b:
        return a
    else:
        return b

In [21]:
def min(a, b):
    if a > b:
        return b
    else:
        return a

In [25]:
maxRating = ratingsRDD.map(lambda entry: entry[2]).reduce(max)
minRating = ratingsRDD.map(lambda entry: entry[2]).reduce(min)

In [33]:
meanRating = float(ratingsRDD.map(lambda entry: entry[2]).sum()) / ratingsRDD.count()

In [46]:
def rescale(entry, mean):
    return entry[0], entry[1], entry[2] - mean

In [47]:
rescaledRaitingsRDD = ratingsRDD.map(lambda entry: rescale(entry, meanRating))

In [49]:
rescaledTrainingRDD = trainingRDD.map(lambda entry: rescale(entry, meanRating))

#### Add Noise

In [57]:
import random

In [64]:
def randomNoise(entry):
    return entry[0], entry[1], entry[2] + 0.2 * random.random()

In [65]:
noisyTrainingRDD = rescaledTrainingRDD.map(lambda entry: randomNoise(entry))

#### Online MC wtih KNN update